In [ ]:
# Cell 1: Header and Setup
import sys
sys.path.insert(0, '..')

from utils.notebook_utils import display_header, display_toc, check_dependency, conclusion_box, info_box
from utils.system_info import display_system_info
from utils.benchmark import Benchmark, BenchmarkResult, ComparisonTable
from utils.charts import setup_style, bar_comparison, throughput_comparison, memory_comparison, COLORS

display_header('Tabular ML Data Comparison', 'SynaDB vs Parquet vs Arrow')

In [ ]:
# Cell 2: Table of Contents
sections = [
    ('Introduction', 'introduction'),
    ('Setup', 'setup'),
    ('Benchmark: Column Selection', 'benchmark-column'),
    ('Benchmark: Aggregations', 'benchmark-aggregation'),
    ('DuckDB Integration', 'duckdb-integration'),
    ('Interoperability Demo', 'interoperability'),
    ('Export/Import Demo', 'export-import'),
    ('Results Summary', 'results'),
    ('Conclusions', 'conclusions'),
]
display_toc(sections)

## 📌 Introduction <a id="introduction"></a>

This notebook compares **SynaDB** against **Parquet** and **Arrow** for tabular ML data:

| System | Type | Key Features |
|--------|------|-------------|
| **SynaDB** | Embedded DB | Single-file, schema-free, native ML support |
| **Parquet** | File Format | Columnar, compressed, widely adopted |
| **Arrow** | In-Memory Format | Zero-copy, cross-language, high performance |

### What We'll Measure

- **Column selection** performance (reading specific features)
- **Aggregation** operations (mean, sum, groupby)
- **DuckDB integration** for SQL queries
- **Interoperability** between formats
- **Export/Import** workflows

### Test Configuration

- **Dataset**: Synthetic ML feature table (1M rows, 50 columns)
- **Column types**: Numeric features, categorical, timestamps

In [ ]:
# Cell 4: System Info
display_system_info()

## 🔧 Setup <a id="setup"></a>

Let's set up our test environment with tabular ML data.

In [ ]:
# Cell 6: Check Dependencies and Imports
import numpy as np
import pandas as pd
import time
import os
import shutil
import tempfile
from pathlib import Path
import matplotlib.pyplot as plt

# Check for SynaDB
HAS_SYNADB = check_dependency('synadb', 'pip install synadb')

# Check for PyArrow (includes Parquet)
HAS_PYARROW = check_dependency('pyarrow', 'pip install pyarrow')

# Check for DuckDB
HAS_DUCKDB = check_dependency('duckdb', 'pip install duckdb')

# Apply consistent styling
setup_style()

In [ ]:
# Cell 7: Generate Tabular ML Data
NUM_ROWS = 100_000  # 100K rows for demo (use 1M for production)
NUM_NUMERIC_COLS = 40
NUM_CATEGORICAL_COLS = 5
SEED = 42

print(f'Generating tabular ML dataset with {NUM_ROWS:,} rows...')
np.random.seed(SEED)

# Generate numeric features
data = {}
for i in range(NUM_NUMERIC_COLS):
    data[f'feature_{i}'] = np.random.randn(NUM_ROWS).astype(np.float32)

# Generate categorical features
categories = ['A', 'B', 'C', 'D', 'E']
for i in range(NUM_CATEGORICAL_COLS):
    data[f'category_{i}'] = np.random.choice(categories, NUM_ROWS)

# Generate target and metadata
data['target'] = np.random.randint(0, 2, NUM_ROWS)
data['timestamp'] = pd.date_range('2024-01-01', periods=NUM_ROWS, freq='s')
data['user_id'] = np.random.randint(1, 10000, NUM_ROWS)

df = pd.DataFrame(data)
print(f'✓ Generated DataFrame with shape: {df.shape}')
print(f'✓ Columns: {len(df.columns)}')
print(f'✓ Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB')

In [ ]:
# Cell 8: Create Temp Directory
temp_dir = tempfile.mkdtemp(prefix='synadb_tabular_')
print(f'Using temp directory: {temp_dir}')

synadb_path = os.path.join(temp_dir, 'features.db')
parquet_path = os.path.join(temp_dir, 'features.parquet')
arrow_path = os.path.join(temp_dir, 'features.arrow')

In [ ]:
# Cell 9: Save Data to SynaDB
synadb_write_time = None

if HAS_SYNADB:
    from synadb import SynaDB
    print('Saving data to SynaDB...')
    start = time.perf_counter()
    with SynaDB(synadb_path) as db:
        # Store each column as a separate key
        for col in df.columns:
            if df[col].dtype == 'float32' or df[col].dtype == 'float64':
                db.put_bytes(f'column/{col}', df[col].values.tobytes())
            elif df[col].dtype == 'int64' or df[col].dtype == 'int32':
                db.put_bytes(f'column/{col}', df[col].values.tobytes())
            elif df[col].dtype == 'datetime64[ns]':
                db.put_bytes(f'column/{col}', df[col].values.astype('int64').tobytes())
            else:
                # Categorical as string
                db.put_text(f'column/{col}', '|'.join(df[col].astype(str)))
        # Store metadata
        db.put_int('metadata/num_rows', NUM_ROWS)
        db.put_text('metadata/columns', '|'.join(df.columns))
    synadb_write_time = time.perf_counter() - start
    print(f'✓ SynaDB: Written in {synadb_write_time:.2f}s')
else:
    print('⚠️ SynaDB not available, skipping...')

In [ ]:
# Cell 10: Save Data to Parquet
parquet_write_time = None

if HAS_PYARROW:
    import pyarrow as pa
    import pyarrow.parquet as pq
    print('Saving data to Parquet...')
    start = time.perf_counter()
    table = pa.Table.from_pandas(df)
    pq.write_table(table, parquet_path, compression='snappy')
    parquet_write_time = time.perf_counter() - start
    print(f'✓ Parquet: Written in {parquet_write_time:.2f}s')
else:
    print('⚠️ PyArrow not available, skipping...')

In [ ]:
# Cell 11: Save Data to Arrow IPC
arrow_write_time = None

if HAS_PYARROW:
    import pyarrow as pa
    import pyarrow.feather as feather
    print('Saving data to Arrow IPC (Feather)...')
    start = time.perf_counter()
    table = pa.Table.from_pandas(df)
    feather.write_feather(table, arrow_path, compression='lz4')
    arrow_write_time = time.perf_counter() - start
    print(f'✓ Arrow IPC: Written in {arrow_write_time:.2f}s')
else:
    print('⚠️ PyArrow not available, skipping...')

In [ ]:
# Cell 12: Write Time Comparison
write_times = {}
if synadb_write_time: write_times['SynaDB'] = synadb_write_time
if parquet_write_time: write_times['Parquet'] = parquet_write_time
if arrow_write_time: write_times['Arrow'] = arrow_write_time

if write_times:
    fig = bar_comparison(write_times, title='Write Time (Tabular Data)', ylabel='Time (seconds)', lower_is_better=True)
    plt.show()

## 📊 Benchmark: Column Selection <a id="benchmark-column"></a>

Let's measure how fast each format can read specific columns - critical for feature selection.

In [ ]:
# Cell 14: Column Selection Setup
# Select 5 random numeric columns
np.random.seed(SEED)
selected_cols = [f'feature_{i}' for i in np.random.choice(NUM_NUMERIC_COLS, 5, replace=False)]
print(f'Testing column selection for: {selected_cols}')

In [ ]:
# Cell 15: SynaDB Column Selection
synadb_col_time = None

if HAS_SYNADB:
    from synadb import SynaDB
    print('Benchmarking SynaDB column selection...')
    
    # Warm up
    with SynaDB(synadb_path) as db:
        for _ in range(3): _ = db.get_bytes(f'column/{selected_cols[0]}')
    
    # Benchmark
    start = time.perf_counter()
    with SynaDB(synadb_path) as db:
        selected_data = {}
        for col in selected_cols:
            col_bytes = db.get_bytes(f'column/{col}')
            selected_data[col] = np.frombuffer(col_bytes, dtype=np.float32)
    synadb_col_time = time.perf_counter() - start
    
    print(f'✓ SynaDB: {len(selected_cols)} columns in {synadb_col_time*1000:.2f}ms')
    print(f'  Total values: {sum(len(v) for v in selected_data.values()):,}')

In [ ]:
# Cell 16: Parquet Column Selection
parquet_col_time = None

if HAS_PYARROW:
    import pyarrow.parquet as pq
    print('Benchmarking Parquet column selection...')
    
    # Warm up
    for _ in range(3): _ = pq.read_table(parquet_path, columns=[selected_cols[0]])
    
    # Benchmark
    start = time.perf_counter()
    table = pq.read_table(parquet_path, columns=selected_cols)
    selected_df = table.to_pandas()
    parquet_col_time = time.perf_counter() - start
    
    print(f'✓ Parquet: {len(selected_cols)} columns in {parquet_col_time*1000:.2f}ms')
    print(f'  Shape: {selected_df.shape}')

In [ ]:
# Cell 17: Arrow Column Selection
arrow_col_time = None

if HAS_PYARROW:
    import pyarrow.feather as feather
    print('Benchmarking Arrow IPC column selection...')
    
    # Warm up
    for _ in range(3): _ = feather.read_table(arrow_path, columns=[selected_cols[0]])
    
    # Benchmark
    start = time.perf_counter()
    table = feather.read_table(arrow_path, columns=selected_cols)
    selected_df = table.to_pandas()
    arrow_col_time = time.perf_counter() - start
    
    print(f'✓ Arrow IPC: {len(selected_cols)} columns in {arrow_col_time*1000:.2f}ms')
    print(f'  Shape: {selected_df.shape}')

In [ ]:
# Cell 18: Column Selection Results
col_times = {}
if synadb_col_time: col_times['SynaDB'] = synadb_col_time * 1000
if parquet_col_time: col_times['Parquet'] = parquet_col_time * 1000
if arrow_col_time: col_times['Arrow'] = arrow_col_time * 1000

if col_times:
    fig = bar_comparison(col_times, title='Column Selection Time (5 columns)', ylabel='Time (ms)', lower_is_better=True)
    plt.show()

## 📈 Benchmark: Aggregations <a id="benchmark-aggregation"></a>

Let's compare aggregation performance - common in feature engineering.

In [ ]:
# Cell 20: SynaDB Aggregation
synadb_agg_time = None

if HAS_SYNADB:
    from synadb import SynaDB
    print('Benchmarking SynaDB aggregation...')
    
    start = time.perf_counter()
    with SynaDB(synadb_path) as db:
        # Load numeric columns and compute aggregations
        results = {}
        for i in range(10):  # First 10 numeric columns
            col = f'feature_{i}'
            col_bytes = db.get_bytes(f'column/{col}')
            arr = np.frombuffer(col_bytes, dtype=np.float32)
            results[col] = {'mean': np.mean(arr), 'std': np.std(arr), 'min': np.min(arr), 'max': np.max(arr)}
    synadb_agg_time = time.perf_counter() - start
    
    print(f'✓ SynaDB: Aggregations on 10 columns in {synadb_agg_time*1000:.2f}ms')

In [ ]:
# Cell 21: Parquet Aggregation
parquet_agg_time = None

if HAS_PYARROW:
    import pyarrow.parquet as pq
    print('Benchmarking Parquet aggregation...')
    
    cols = [f'feature_{i}' for i in range(10)]
    start = time.perf_counter()
    table = pq.read_table(parquet_path, columns=cols)
    df_subset = table.to_pandas()
    results = df_subset.agg(['mean', 'std', 'min', 'max'])
    parquet_agg_time = time.perf_counter() - start
    
    print(f'✓ Parquet: Aggregations on 10 columns in {parquet_agg_time*1000:.2f}ms')

In [ ]:
# Cell 22: Arrow Aggregation
arrow_agg_time = None

if HAS_PYARROW:
    import pyarrow.feather as feather
    import pyarrow.compute as pc
    print('Benchmarking Arrow aggregation...')
    
    cols = [f'feature_{i}' for i in range(10)]
    start = time.perf_counter()
    table = feather.read_table(arrow_path, columns=cols)
    # Use Arrow compute for aggregations
    results = {}
    for col in cols:
        arr = table.column(col)
        results[col] = {
            'mean': pc.mean(arr).as_py(),
            'min': pc.min(arr).as_py(),
            'max': pc.max(arr).as_py()
        }
    arrow_agg_time = time.perf_counter() - start
    
    print(f'✓ Arrow: Aggregations on 10 columns in {arrow_agg_time*1000:.2f}ms')

In [ ]:
# Cell 23: Aggregation Results
agg_times = {}
if synadb_agg_time: agg_times['SynaDB'] = synadb_agg_time * 1000
if parquet_agg_time: agg_times['Parquet'] = parquet_agg_time * 1000
if arrow_agg_time: agg_times['Arrow'] = arrow_agg_time * 1000

if agg_times:
    fig = bar_comparison(agg_times, title='Aggregation Time (10 columns)', ylabel='Time (ms)', lower_is_better=True)
    plt.show()

## 🦆 DuckDB Integration <a id="duckdb-integration"></a>

DuckDB can query Parquet files directly with SQL. Let's compare this workflow.

In [ ]:
# Cell 25: DuckDB Setup
if HAS_DUCKDB:
    import duckdb
    print('DuckDB Integration Demo\n')
    print('DuckDB can query Parquet files directly with SQL!')
else:
    print('⚠️ DuckDB not available, skipping integration demo...')

In [ ]:
# Cell 26: DuckDB Query on Parquet
duckdb_query_time = None

if HAS_DUCKDB and HAS_PYARROW:
    import duckdb
    print('Benchmarking DuckDB SQL query on Parquet...')
    
    # Complex aggregation query
    query = f'''
    SELECT 
        category_0,
        COUNT(*) as count,
        AVG(feature_0) as avg_f0,
        AVG(feature_1) as avg_f1,
        SUM(target) as total_positive
    FROM read_parquet('{parquet_path}')
    GROUP BY category_0
    ORDER BY count DESC
    '''
    
    # Warm up
    for _ in range(3): _ = duckdb.query(query).fetchall()
    
    # Benchmark
    start = time.perf_counter()
    result = duckdb.query(query).fetchdf()
    duckdb_query_time = time.perf_counter() - start
    
    print(f'✓ DuckDB: Complex aggregation in {duckdb_query_time*1000:.2f}ms')
    print(f'\nResult:')
    print(result)

In [ ]:
# Cell 27: Equivalent SynaDB + Pandas Query
synadb_query_time = None

if HAS_SYNADB:
    from synadb import SynaDB
    print('Benchmarking equivalent SynaDB + Pandas query...')
    
    start = time.perf_counter()
    with SynaDB(synadb_path) as db:
        # Load required columns
        cat0_str = db.get_text('column/category_0')
        cat0 = cat0_str.split('|')
        f0 = np.frombuffer(db.get_bytes('column/feature_0'), dtype=np.float32)
        f1 = np.frombuffer(db.get_bytes('column/feature_1'), dtype=np.float32)
        target = np.frombuffer(db.get_bytes('column/target'), dtype=np.int64)
        
        # Create DataFrame and aggregate
        query_df = pd.DataFrame({'category_0': cat0, 'feature_0': f0, 'feature_1': f1, 'target': target})
        result = query_df.groupby('category_0').agg(
            count=('feature_0', 'count'),
            avg_f0=('feature_0', 'mean'),
            avg_f1=('feature_1', 'mean'),
            total_positive=('target', 'sum')
        ).sort_values('count', ascending=False)
    synadb_query_time = time.perf_counter() - start
    
    print(f'✓ SynaDB + Pandas: Complex aggregation in {synadb_query_time*1000:.2f}ms')
    print(f'\nResult:')
    print(result)

In [ ]:
# Cell 28: Query Comparison
query_times = {}
if synadb_query_time: query_times['SynaDB + Pandas'] = synadb_query_time * 1000
if duckdb_query_time: query_times['DuckDB + Parquet'] = duckdb_query_time * 1000

if query_times:
    fig = bar_comparison(query_times, title='Complex Aggregation Query', ylabel='Time (ms)', lower_is_better=True)
    plt.show()

## 🔄 Interoperability Demo <a id="interoperability"></a>

Let's demonstrate how data can flow between these formats.

In [ ]:
# Cell 30: Interoperability Demo
print('Interoperability Demonstration\n')
print('=' * 70)

print('\n📦 SynaDB Interoperability:')
print('  - Export to NumPy arrays (native)')
print('  - Export to Pandas DataFrame (via column loading)')
print('  - Import from any format that produces bytes/arrays')
print('  - Best for: ML pipelines with mixed data types')

print('\n📦 Parquet Interoperability:')
print('  - Native Arrow integration')
print('  - Direct DuckDB/Spark/Pandas support')
print('  - Industry standard for data lakes')
print('  - Best for: Data engineering pipelines')

print('\n📦 Arrow Interoperability:')
print('  - Zero-copy sharing between processes')
print('  - Cross-language support (Python, R, Julia, etc.)')
print('  - Native Parquet read/write')
print('  - Best for: High-performance data exchange')

print('\n' + '=' * 70)

In [ ]:
# Cell 31: SynaDB to Pandas Demo
if HAS_SYNADB:
    from synadb import SynaDB
    print('SynaDB → Pandas DataFrame Demo\n')
    
    with SynaDB(synadb_path) as db:
        # Load specific columns
        cols_to_load = ['feature_0', 'feature_1', 'feature_2', 'target']
        data = {}
        for col in cols_to_load:
            col_bytes = db.get_bytes(f'column/{col}')
            if col == 'target':
                data[col] = np.frombuffer(col_bytes, dtype=np.int64)
            else:
                data[col] = np.frombuffer(col_bytes, dtype=np.float32)
        
        result_df = pd.DataFrame(data)
        print(f'Loaded DataFrame shape: {result_df.shape}')
        print(result_df.head())

In [ ]:
# Cell 32: Arrow Zero-Copy Demo
if HAS_PYARROW:
    import pyarrow as pa
    import pyarrow.feather as feather
    print('Arrow Zero-Copy Demo\n')
    
    # Read Arrow file
    table = feather.read_table(arrow_path)
    
    # Zero-copy to NumPy
    col = table.column('feature_0')
    np_array = col.to_numpy(zero_copy_only=False)  # May need copy for chunked arrays
    
    print(f'Arrow column type: {col.type}')
    print(f'NumPy array shape: {np_array.shape}')
    print(f'NumPy array dtype: {np_array.dtype}')
    print(f'\nFirst 5 values: {np_array[:5]}')

## 📤 Export/Import Demo <a id="export-import"></a>

Let's demonstrate exporting SynaDB data to Parquet and vice versa.

In [ ]:
# Cell 34: SynaDB to Parquet Export
export_time = None

if HAS_SYNADB and HAS_PYARROW:
    from synadb import SynaDB
    import pyarrow as pa
    import pyarrow.parquet as pq
    
    print('Exporting SynaDB → Parquet...')
    export_path = os.path.join(temp_dir, 'exported.parquet')
    
    start = time.perf_counter()
    with SynaDB(synadb_path) as db:
        # Load numeric columns
        export_data = {}
        for i in range(5):  # First 5 features
            col = f'feature_{i}'
            col_bytes = db.get_bytes(f'column/{col}')
            export_data[col] = np.frombuffer(col_bytes, dtype=np.float32)
        
        # Create Arrow table and write Parquet
        table = pa.Table.from_pydict(export_data)
        pq.write_table(table, export_path)
    export_time = time.perf_counter() - start
    
    export_size = os.path.getsize(export_path) / (1024 * 1024)
    print(f'✓ Exported in {export_time*1000:.2f}ms')
    print(f'  Output size: {export_size:.2f} MB')

In [ ]:
# Cell 35: Parquet to SynaDB Import
import_time = None

if HAS_SYNADB and HAS_PYARROW:
    from synadb import SynaDB
    import pyarrow.parquet as pq
    
    print('Importing Parquet → SynaDB...')
    import_db_path = os.path.join(temp_dir, 'imported.db')
    
    start = time.perf_counter()
    # Read Parquet
    table = pq.read_table(parquet_path, columns=['feature_0', 'feature_1', 'target'])
    
    # Write to SynaDB
    with SynaDB(import_db_path) as db:
        for col_name in table.column_names:
            col = table.column(col_name)
            arr = col.to_numpy()
            db.put_bytes(f'column/{col_name}', arr.tobytes())
    import_time = time.perf_counter() - start
    
    import_size = os.path.getsize(import_db_path) / (1024 * 1024)
    print(f'✓ Imported in {import_time*1000:.2f}ms')
    print(f'  Output size: {import_size:.2f} MB')

In [ ]:
# Cell 36: Export/Import Summary
print('Export/Import Summary\n')
print('=' * 50)
if export_time:
    print(f'SynaDB → Parquet: {export_time*1000:.2f}ms')
if import_time:
    print(f'Parquet → SynaDB: {import_time*1000:.2f}ms')
print('\n✓ Seamless data exchange between formats!')

## 📊 Results Summary <a id="results"></a>

Let's summarize all benchmark results.

In [ ]:
# Cell 38: Results Summary Table
from IPython.display import display, Markdown

# Build summary table
summary_md = '''\n| Metric | SynaDB | Parquet | Arrow |\n|--------|--------|---------|-------|\n'''

# Write time
synadb_wt = f'{synadb_write_time:.2f}s' if synadb_write_time else 'N/A'
parquet_wt = f'{parquet_write_time:.2f}s' if parquet_write_time else 'N/A'
arrow_wt = f'{arrow_write_time:.2f}s' if arrow_write_time else 'N/A'
summary_md += f'| Write Time | {synadb_wt} | {parquet_wt} | {arrow_wt} |\n'

# Column selection
synadb_cs = f'{synadb_col_time*1000:.2f}ms' if synadb_col_time else 'N/A'
parquet_cs = f'{parquet_col_time*1000:.2f}ms' if parquet_col_time else 'N/A'
arrow_cs = f'{arrow_col_time*1000:.2f}ms' if arrow_col_time else 'N/A'
summary_md += f'| Column Selection | {synadb_cs} | {parquet_cs} | {arrow_cs} |\n'

# Aggregation
synadb_ag = f'{synadb_agg_time*1000:.2f}ms' if synadb_agg_time else 'N/A'
parquet_ag = f'{parquet_agg_time*1000:.2f}ms' if parquet_agg_time else 'N/A'
arrow_ag = f'{arrow_agg_time*1000:.2f}ms' if arrow_agg_time else 'N/A'
summary_md += f'| Aggregation | {synadb_ag} | {parquet_ag} | {arrow_ag} |\n'

# Features
summary_md += '| SQL Support | Via Pandas | ✓ DuckDB | ✓ DuckDB |\n'
summary_md += '| Schema-Free | ✓ Yes | ✗ No | ✗ No |\n'
summary_md += '| Single File | ✓ Yes | ✓ Yes | ✓ Yes |\n'
summary_md += '| Compression | ✓ LZ4 | ✓ Snappy/Zstd | ✓ LZ4/Zstd |\n'

display(Markdown(summary_md))

In [ ]:
# Cell 39: Storage Size Comparison
def get_file_size(path):
    if os.path.exists(path):
        return os.path.getsize(path) / (1024 * 1024)
    return 0

file_sizes = {}
if os.path.exists(synadb_path): file_sizes['SynaDB'] = get_file_size(synadb_path)
if os.path.exists(parquet_path): file_sizes['Parquet'] = get_file_size(parquet_path)
if os.path.exists(arrow_path): file_sizes['Arrow'] = get_file_size(arrow_path)

if file_sizes:
    fig = memory_comparison(file_sizes, title='Storage Size Comparison', ylabel='Size (MB)')
    plt.show()
    print('\nFile sizes:')
    for name, size in file_sizes.items():
        print(f'  {name}: {size:.1f} MB')

## 🎯 Conclusions <a id="conclusions"></a>

In [ ]:
# Cell 41: Conclusions
conclusion_box(
    title='Key Takeaways',
    points=[
        'Parquet excels at columnar analytics with excellent compression',
        'Arrow provides fastest in-memory operations with zero-copy',
        'SynaDB offers schema flexibility - mix data types without migration',
        'DuckDB + Parquet is powerful for SQL analytics on files',
        'SynaDB is ideal for ML workflows with evolving schemas',
        'All formats support efficient column selection',
    ],
    summary='Choose based on workflow: Parquet for data lakes and analytics, Arrow for high-performance exchange, SynaDB for flexible ML data management.'
)

In [ ]:
# Cell 42: Cleanup
import shutil
try:
    shutil.rmtree(temp_dir)
    print(f'✓ Cleaned up temp directory: {temp_dir}')
except Exception as e:
    print(f'⚠️ Could not clean up: {e}')